
### Problem Statement
The proliferation of online content necessitates effective methods for detecting and moderating harmful language. This project addresses the challenge of classifying online comments. This is formulated as a **multi-label classification problem**, where each comment can belong to zero, one toxicity categories.

### Motivation
Automated toxic comment detection is crucial for maintaining healthy online communities, preventing cyberbullying, and reducing the burden on human moderators. Building accurate and robust models contributes significantly to platform safety and user well-being.



### Libraries
We will primarily use the following libraries:
- `pandas` for data manipulation.
- `numpy` for numerical operations.
- `scikit-learn` for splitting data and evaluation metrics.
- `torch` (or `tensorflow`) as the deep learning framework.
- `transformers` by Hugging Face for accessing pre-trained models (BERT) and tokenizers.


---

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers


In [ ]:
!pip install transformers datasets scikit-learn torch


In [5]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Toxic_comment/train.csv')
df = data[['comment_text', 'toxic']]  # Use 'data' instead of 'df' to select columns
df['toxic'] = df['toxic'].astype(int)
df.head(28)

<ipython-input-5-396440c69988>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['toxic'] = df['toxic'].astype(int)


,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [6]:
pip install transformers torch


In [7]:
from transformers import pipeline

classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
results = classifier("You are so dumb..")
print(results)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'NEGATIVE', 'score': 0.9996798038482666}]


In [14]:
import torch # Import the torch library

In [15]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
# Load the pretrained T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("ramsrigouthamg/t5_paraphraser")
tokenizer = T5Tokenizer.from_pretrained("ramsrigouthamg/t5_paraphraser")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Optimized paraphrasing function
def paraphrase_text(input_text, max_length=256, num_return_sequences=1, num_beams=10):
    prompt = f"paraphraser: {input_text} </s>"
    input_ids = tokenizer.encode(prompt, return_tensors="pt", truncation=True).to(device)

    # Tokenize
    encoding = tokenizer(prompt, padding="max_length", max_length=256, truncation=True, return_tensors="pt")

    # Generate paraphrased text
    outputs = model.generate(
        input_ids=encoding["input_ids"],
        attention_mask=encoding["attention_mask"],
        max_length=256,
        do_sample=True,
        top_k=120,
        top_p=0.98,
        early_stopping=True,
        num_return_sequences=1
    )

    # Decode and return
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [16]:
# check the comment is toxic or not if comment is toxic then moderaized the comment if it is not toxic  then  comment make as it same toxic_comments = [


toxic_comments = [
    "You're so dumb!",
    "Nobody cares about your opinion.",
    "You're a complete waste of space.",
    'You freaking suck! I am going to hit you.',
    'You are a wonderful person!',
    'This is so stupid and annoying!',
    'You are so disgusting and cheep',
    'wow! you look  damm beautiful',
    "You're so dumb, it's honestly painful to read your posts.",
    "Nobody cares about your opinion, just shut up already.",
    "You're a complete waste of space.",
    "This is the stupidest thing I've ever read.",
    "Go crawl back into whatever hole you came from.",
    "Your existence is a joke.",
    "Only an idiot would think like this.",
    "You're a disgusting human being.",
    "I really enjoyed your post, thanks for sharing!",
    "This is such a thoughtful perspective.",
    "You explained it so clearly, well done!",
    "I'm learning a lot from your content.",
    "Great job on this project, it's impressive.",
    "Thank you for the detailed response.",
    "I appreciate the respectful discussion.",
    "This is a very informative article.",
    "You made a strong argument with evidence.",
    "Keep up the great work!",
    "You're so stupid, how can anyone listen to you?",
    "This is the dumbest thing I've ever read.",
    "Shut up, no one cares about your opinion.",
    "You sound like a complete moron.",
    "You're a total waste of space.",
    "Idiots like you ruin everything.",
    "You're a joke and nobody respects you.",
    "Go cry somewhere else, loser.",
    "Take your nonsense elsewhere, idiot.",
    "You're full of crap.",
    "Your brain must be mush.",
    "Go touch grass, freak.",
    "Can't believe how toxic and dumb you are.",
    "You're too dense to understand anything.",
    "This is the type of idiocy that breaks the internet."
]
for comment in toxic_comments:
    results = classifier(comment)
    print("Original comment:", comment)
    if results[0]['label'] == 'NEGATIVE':
        print("Moderated comment:", modernize_text(comment))
    else:
        print("Comment:", comment)
    print("---")


Original comment: You're so dumb!


/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Moderated comment: Is it possible for you to be stupid?
---
Original comment: Nobody cares about your opinion.
Moderated comment: I cannot care about someone's opinion on anything. Is it a big deal?
---
Original comment: You're a complete waste of space.
Moderated comment: You're a terrible waste of space. You're an idiot.
---
Original comment: You freaking suck! I am going to hit you.
Comment: You freaking suck! I am going to hit you.
---
Original comment: You are a wonderful person!
Comment: You are a wonderful person!
---
Original comment: This is so stupid and annoying!
Moderated comment: What is it about being annoying?
---
Original comment: You are so disgusting and cheep
Moderated comment: You are so disgusting and cheep.
---
Original comment: wow! you look  damm beautiful
Comment: wow! you look  damm beautiful
---
Original comment: You're so dumb, it's honestly painful to read your posts.
Moderated comment: I find you dumb so it's very painful to read your blog posts.
---
Origi

In [11]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00


In [18]:
# prompt: create a gui in which type a toxic comment by user and generate the non toix comment
#  and  a create a page in another window

import gradio as gr # Import the gradio library and assign it to the alias 'gr'


def process_comment(comment):
    results = classifier(comment)
    if results[0]['label'] == 'NEGATIVE':
        moderated_comment = modernize_text(comment)
        return moderated_comment
    else:
        return comment


iface = gr.Interface(
    fn=process_comment,
    inputs=gr.Textbox(lines=2, placeholder="Enter a comment here..."),
    outputs="text",
    title="Toxic Comment Moderator",
    description="Enter a comment, and the system will moderate it if toxic."
)

iface.launch(share=True) # Set share=True to get a public link

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://82bced610c24ce5e93.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
